In [2]:
import pandas as pd
import numpy as np
%pylab inline
import seaborn

Populating the interactive namespace from numpy and matplotlib


In [3]:
df = pd.read_csv("/data/home/taras.svirsky/lab12/data/stest_pivot4ensemble.csv")

In [4]:
df.shape

(3159915, 17)

In [5]:
#sum(df.isnull())

In [6]:
def get_rmse(sdf, column):
    e = sdf['rating'] - sdf[column]
    return np.sqrt(np.mean(e*e))

In [7]:
for f in ['bp_rating', 'predicted_i2i_rating', 'predicted_u2u_rating', 'predicted_als_rating']:
    print f, get_rmse(df, f)

bp_rating 0.886703322587
predicted_i2i_rating 0.92077309331
predicted_u2u_rating 0.813953247318
predicted_als_rating 0.837009799107


In [8]:
df['pred'] = 0.75 * df["predicted_als_rating"] + 0.25 * df["predicted_i2i_rating"]

In [9]:
get_rmse(df,'pred')

0.82330544521203797

In [10]:
recs = []
for j in range(20):
    for i in range(20):
        r1 = 0.05*i
        r2 = 0.05*j
        r3 = 1 - r1 - r2
        if r3 < 0:
            continue
        df['pred'] = r1 * df["predicted_als_rating"] + r2 * df["predicted_u2u_rating"] + r3 * df["predicted_i2i_rating"]
        recs.append((r1,r2,r3, get_rmse(df,'pred')))
        
sdf = pd.DataFrame.from_records(recs, columns=["als", "u2u", "i2i", "rmse"])        

In [11]:
sdf.sort_values(by="rmse").head()

,als,u2u,i2i,rmse
173,0.30,0.55,0.15,0.786940
163,0.35,0.50,0.15,0.787097
174,0.35,0.55,0.10,0.787246
182,0.25,0.60,0.15,0.787478
183,0.30,0.60,0.10,0.787550


In [12]:
# looks like we have overfit on u2u, because it gives 0.86 rmse on submission

In [13]:
# so I think to use second row

In [14]:
#als 0.35 u2u 0.50 i2i 0.15
#als 0.6  u2u 0.3  i2i 0.1
#als 0.50 u2u 0.35 i2i 0.15

# just for fun

In [52]:
from sklearn.cross_validation import cross_val_score, train_test_split

In [60]:
from sklearn.cross_validation import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [79]:
lr = LinearRegression(fit_intercept=False)

In [226]:
idx_big = (df.n_user_ratings > 50) & (df.n_item_ratings>100)
idx_med = (df.n_user_ratings > 5) & (df.n_user_ratings <= 50) & (df.n_item_ratings>100)

In [549]:
idx = (~idx_med) & (~idx_big)

In [550]:
df['i2i_x_als'] = df['predicted_als_rating']*df['predicted_i2i_rating']

In [577]:
X = df[idx][["predicted_als_rating", "predicted_i2i_rating", "i2i_x_als"]]

In [578]:
y = df[idx]['rating']

In [579]:
cv = KFold(len(df[idx]), n_folds=10, shuffle=True, random_state=17)

In [580]:
z=cross_val_score(lr, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=4)

In [581]:
-np.mean(z)

0.90334973544281882

In [582]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=434)

In [583]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [584]:
lr.coef_

array([ 0.48310744,  0.64862636, -0.03451447])

In [585]:
mean_squared_error(y_test, lr.predict(X_test))

0.91593672096972778

In [586]:
lr.intercept_

0.0

In [587]:
# big -0.35258075,  1.23212959,  0.15828964
# med 

In [ ]:
array([ 0.48307975,  0.65027222, -0.03491632])